In [2]:
import  pandas as pd
import numpy as np
df = pd.read_csv('merged_df_abstract_preprocess_all.csv')
df = df.dropna(subset=['processed_abstract'])

C:\Users\cshch\AppData\Local\Temp\ipykernel_6388\2488393254.py:3: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('merged_df_abstract_preprocess_all.csv')


In [4]:
print(df.shape)
print(df.columns)

(311557, 20)
Index(['Title', 'Abstract', 'Disciplines', 'Keywords', 'Source',
       'Disciplines_split', 'Astronomy', 'Biology', 'Chemistry',
       'Computer Science', 'Economics', 'Education & instruction',
       'Electrical Engineering', 'European & international law', 'History',
       'Mathematics', 'Physics', 'Social Sciences', 'Statistics',
       'processed_abstract'],
      dtype='object')


In [8]:
# !pip install --upgrade scikit-learn imbalanced-learn

  Attempting uninstall: imbalanced-learn
    Found existing installation: imbalanced-learn 0.12.2
    Uninstalling imbalanced-learn-0.12.2:
      Successfully uninstalled imbalanced-learn-0.12.2


In [3]:
discipline_mapping = {
    "Astronomy": "Physics",
    "Astronomy/Physics": "Physics",
    "Biology": "Biology",
    "Chemistry/Physics": "Physics",
    "Computer Science": "Computer Science",
    "Computer Science/Electrical Engineering": "Computer Science",
    "Computer Science/Mathematics": "Mathematics",
    "Computer Science/Statistics": "Mathematics",
    "Economics": "Social Sciences",
    "Education & instruction": "Social Sciences",
    "Electrical Engineering": "Electrical Engineering",
    "European & international law": "Social Sciences",
    "History": "Social Sciences",
    "Mathematics": "Mathematics",
    "Mathematics/Physics": "Physics",
    "Mathematics/Statistics": "Mathematics",
    "Physics": "Physics",
    "Social Sciences": "Social Sciences",
    "Statistics": "Mathematics",
    "Computer Science/Electrical Engineering/Mathematics": "Mathematics"
}
# Apply the discipline mapping to the 'Disciplines' column in the DataFrame
df['Disciplines_merged'] = df['Disciplines'].map(discipline_mapping)

# Check the changes
print(df['Disciplines_merged'].value_counts())

Disciplines_merged
Computer Science          133914
Physics                    96677
Mathematics                68196
Social Sciences             5666
Electrical Engineering      5583
Biology                     1521
Name: count, dtype: int64


In [5]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

# Vectorize the Text Data using TF-IDF
vectorizer_Logistic_MultiDiscipline = TfidfVectorizer()
X = vectorizer_Logistic_MultiDiscipline.fit_transform(df['processed_abstract'])
y = df['Disciplines_merged']

# Split the data into training and testing sets (without oversampling)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Training set size:", X_train.shape)
print("Test set size:", X_test.shape)

Training set size: (249245, 106847)
Test set size: (62312, 106847)


In [6]:
# 2 MLP
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, classification_report

# Initialize the MLPClassifier with desired parameters
model_MLP = MLPClassifier(hidden_layer_sizes=(100,), max_iter=1000, activation='relu', solver='adam')

# Train the model on the training data (no oversampling, just the original split)
model_MLP.fit(X_train, y_train)

# Make predictions on the test data
y_pred = model_MLP.predict(X_test)

# Evaluate the model's performance
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))


C:\Users\cshch\AppData\Roaming\Python\Python311\site-packages\sklearn\neural_network\_multilayer_perceptron.py:697: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


Accuracy: 0.9204808062652459
Classification Report:
                         precision    recall  f1-score   support

               Biology       0.85      0.73      0.79       292
      Computer Science       0.94      0.94      0.94     26884
Electrical Engineering       0.47      0.46      0.46      1079
           Mathematics       0.88      0.88      0.88     13638
               Physics       0.96      0.96      0.96     19229
       Social Sciences       0.78      0.75      0.76      1190

              accuracy                           0.92     62312
             macro avg       0.81      0.79      0.80     62312
          weighted avg       0.92      0.92      0.92     62312



In [7]:
# Save model
import joblib
import os

# Define the directory to save the model
save_dir = 'model'
os.makedirs(save_dir, exist_ok=True)

# Save the MLP model
model_file_path = os.path.join(save_dir, 'Model_MLP.joblib')
joblib.dump(model_MLP, model_file_path)

# Load the model back
# loaded_model = joblib.load(model_file_path)
# print("Model successfully loaded:", loaded_model)


['model\\Model_MLP.joblib']

In [15]:
import pandas as pd

n_top_keywords = 100
feature_names = vectorizer_Logistic_MultiDiscipline.get_feature_names_out()

keywords_dict = {}

for i, label in enumerate(model_Logistic_MultiDiscipline_oversampling.classes_):
    coef = model_Logistic_MultiDiscipline_oversampling.coef_[i]
    top_keywords_idx = coef.argsort()[-n_top_keywords:][::-1]

    top_keywords = [(feature_names[idx], coef[idx]) for idx in top_keywords_idx]
    keywords_dict[label] = top_keywords

keywords_df = pd.DataFrame.from_dict(keywords_dict, orient='index')
keywords_df.to_csv("keywords_from_model/2_keywords_Logistic_oversampling.csv")
